#Install libraries

In [ ]:
%%capture
!pip install --pre pycaret
!pip install catboost

#Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import libraries

In [ ]:
import catboost
from catboost import CatBoostClassifier

import pycaret
from pycaret.classification import *
from pycaret.regression import load_model 
from pycaret.classification import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import pandas as pd
import numpy as np
import datetime

import pickle
import joblib
from joblib import Parallel, delayed




pd.set_option("display.max_columns",None )

#Process Class, batch saving

In [ ]:
class Tourists:
    
    df_tourists = pd.DataFrame([])
    df_uselesses = pd.DataFrame([])
    
    df_tourists_columns = df_tourists.columns
    df_uselesses_columns = df_uselesses.columns
    
    train = pd.DataFrame([])
    test = pd.DataFrame([])
    train_label = []
    test_label = []
    df_Total =  pd.DataFrame([])
    df_Total_processed = []
    df_Total_processed_label = []
    df_Total_labels = []
    
    categorical_feature_list = []
    prediction = []
    Probabilities = []
    Catboost_model = []
    tn=0
    fp=0
    fn=0
    tp=0
    Accuracy = -1
    Recall =0 
    Precision =0
    Error_rate =0
    adress_convert=''
    adress_notconvert=''
    
    def __init__(self,adress_convert,adress_notconvert):
        print("Hi Tourists class has been initialized!")
        
        self.df_tourists = pd.read_csv(adress_convert)
        self.df_uselesses = pd.read_csv(adress_notconvert)
        self.df_Total = pd.concat([self.df_tourists, self.df_uselesses])
        self.df_Total_processed_label = np.concatenate((np.ones(self.df_tourists.shape[0]), 
                                                        np.zeros(self.df_uselesses.shape[0])), axis=None)
        self.df_Total_processed = self.preprocessf(self.df_Total)
        self.df_tourists['label'] = 1
        self.df_uselesses['label'] = 0

        self.df_Total_labels = self.df_Total_processed
        self.df_Total_labels['labels'] = self.df_Total_processed_label
        self.df_Total_labels = self.df_Total_labels.sample(frac = 1).reset_index()

        
        # self.batch_train_dataset(self.df_Total_labels, 5,'save')


    def One_Hot_Enc(self, df, column):
        
        services = pd.get_dummies(df[column].str.split(',', expand=True),
                                  prefix=column, prefix_sep='').groupby(level=0, axis=1).max()
        df = pd.concat([df, services], axis='columns')
        return df
    
        
    def preprocessf(self, df):
        
        #pivote on services
        columns = ['senfs', 'serviceusage', 'topservices' ,'topservicesexc2c', 'businesstags', 'bankscp']
        for x in columns:
            df = self.One_Hot_Enc(df, x)
            
         # numerical sex   
        cleanup_nums = {"gender": {"Male": 1, "Female": 2}}
        df = df.replace(cleanup_nums)  
            
        #fillna
        for x in df.columns:
               df[x] = df[x].fillna(0)
        
         
        # len feature
        list_len = ['topservices', 'topservicesexc2c', 'businesstags', 'serviceusage', 'senfs', 'bankscp']
        for x in list_len:
            df['len_'+x] = df[x].apply(lambda x: len(str(x).split(',')))
        
        # Age feature
        today = datetime.date.today()
        this_year = int(today.strftime("%Y"))
        
        list_age = ['firsttrsdateapplication', 'firstappinstalldate', 'birthday']
        for x in list_age:
            df['age_'+x] = this_year - df[x].str.split('T',expand = True)[0].str.split('-',expand = True)[0].fillna(0).apply(int)
            
        #drop extra features
        df = df.drop(columns = ['Unnamed: 0',
                                'uuid', 'ussdid', 
                                'email', #'name', 
                                'birthday', 'bankscp',
                                'businesstags',        
                                'firsttrsdateapplication', 'firstappinstalldate',
                                'firstprovince', #'firstregion','secondprovince', 'secondregion',
                                'maskedmobilenumber', 'maskednationalid',
                                'senfs', 'serviceusage', 'topservices', 'topservicesexc2c'])
        #integer
        cat = ['name','firstregion','secondprovince', 'secondregion']
        
        for x in [x for x in df.columns if (x not in cat)]:
               df[x] = df[x].apply(int)
        
        return df
    
    
    def split_test(self, df_X, df_Y):
        
        self.train, self.test, self.train_label, self.test_label = train_test_split(df_X, df_Y, 
                                                                                     random_state=104, 
                                                                                     test_size=0.25, shuffle=True)  
        
        return (self.train, self.test, self.train_label, self.test_label)
        

    def Evaluation(self):

        self.tn, self.fp, self.fn, self.tp =  confusion_matrix(self.prediction, self.test_label).ravel() 
        self.Recall =self.tp / (self.tp + self.fn)
        self.Precision = self.tp / (self.tp + self.fp)
        self.Error_rate = (self.fp+self.fn)/(self.tp+self.tn+self.fp+self.fn)
        self.Accuracy = (self.tp+self.tn)/(self.tp+self.tn+self.fp+self.fn)
        print("\nAccuracy", self.Accuracy)
        print("\nError_rate", self.Error_rate)
        print("\nRecall", self.Recall)
        print("\nPrecision", self.Precision)
        return 0
    
    def prediction_model(self):
            
        self.split_test(self.df_Total_processed, self.df_Total_processed_label)    
        
        categorical_feature_list = [ self.train.columns.get_loc('name'), self.train.columns.get_loc('firstregion'), 
                                    self.train.columns.get_loc('secondprovince'), self.train.columns.get_loc('secondregion')] 
           
        # Initialize CatBoostClassifier
        model = CatBoostClassifier(iterations=2,
                                   learning_rate=1,
                                   depth=2)
        
        # Fit model
        self.Catboost_model = model.fit(self.train, self.train_label, categorical_feature_list)
                                        #verbose=False,plot=True)
        
        self.Catboost_model.save_model('catboost') 

        # Get predicted classes
        self.prediction = model.predict(self.test)
        
        # Get predicted probabilities for each class
        self.Probabilities = model.predict_proba(self.test)
        
        # Get predicted RawFormulaVal
        preds_raw = model.predict(self.test, prediction_type='RawFormulaVal') 
        
        self.Evaluation()
        return model


    def Pycaret_model(self):
      
        self.df_total = df_total.sample(frac = 1).reset_index()
        # prediction_model. 
        
       # Save the model as a pickle in a file
        joblib.dump(knn, 'filename.pkl')
  
        # Load the model from the file
        knn_from_joblib = joblib.load('filename.pkl')
  
        # Use the loaded model to make predictions
        knn_from_joblib.predict(X_test)    

        return 0
    
    def batch_train_dataset(self, dataframe, number_of_batch, task):

      batch_size = int(dataframe.shape[0]/number_of_batch)

      if task == 'save':
        for i in range (1, number_of_batch+1):
          name = 'PotentialUser_' + i.__str__() + '_Batch' + '.csv'
          dataframe[(i-1)*batch_size: i*batch_size].to_csv("/content/drive/My Drive/" + name, index=False)

      if task == 'load':
        for i in range (1, number_of_batch+1):
          name = r'PotentialUser_' + i.__str__() + '_Batch' + '.csv'
          globals()[f"dataframe_batch{i}"] = pd.read_csv("/content/drive/My Drive/" + name)
      
      return 0    

#Main

In [ ]:
 if __name__ == '__main__':  
   
  a='/content/drive/My Drive/converted_join_cp.csv'
  b='/content/drive/My Drive/notconverted_join_cp.csv'

  Class = Tourists(a,b)
  df = Class.df_Total_labels

Hi Tourists class has been initialized!


#Pycaret Setups

https://pycaret.gitbook.io/docs/get-started/preprocessing/feature-engineering

In [ ]:
s = setup(globals()[f"dataframe_batch{1}"].drop(columns =['labels']) ,
          target = globals()[f"dataframe_batch{1}"]['labels'] ,
          fix_imbalance = True ,
          remove_outliers = True ,
          normalize = True , 
          normalize_method = 'robust' ,
          bin_numeric_features = ['age_birthday'] ,
          feature_selection = True ,
          remove_multicollinearity  = True , 
          pca = True , 
          use_gpu = True , 
          profile = True) 

In [ ]:
best = compare_models()

In [ ]:
plot_model(best)

In [ ]:
evaluate_model(best)

In [ ]:
from pycaret.regression import load_model 

Ridge_Classifier = create_model('ridge') 
save_model(Ridge_Classifier, 'saved_ridge_model') 
saved_ridge = load_model('saved_ridge_model') 

#Catboost

https://catboost.ai/en/docs/features/visualization_jupyter-notebook


In [ ]:
df_X = df.drop(columns = ['labels'])
df_Y = df['labels']
train, test, train_label, test_label = train_test_split(df_X, df_Y,
                                                        random_state=104, 
                                                        test_size=0.25,
                                                        shuffle=True)  
        
categorical_feature_list = [ train.columns.get_loc('name'), train.columns.get_loc('firstregion'), 
                              train.columns.get_loc('secondprovince'), train.columns.get_loc('secondregion')] 
           
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=2,
                          learning_rate=1,
                          depth=2)
        
# Fit model
Catboost_model = model.fit(df_X, df_Y, categorical_feature_list)
                                        #verbose=False,plot=True)
        
Catboost_model.save_model('/content/drive/My Drive/catboost',
           format="cbm",
           export_parameters=None,
           pool=None)

# # Get predicted classes
prediction = model.predict(test)
        
# # Get predicted probabilities for each class
# Probabilities = model.predict_proba(test)

In [ ]:
cm = confusion_matrix(predictions['prediction_label'], Y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=[0,1])
disp.plot()

#Evaluation

##Quadratic Discriminant Analysis

In [ ]:
y_true = Y_test
y_pred = predictions_qda['prediction_label']
target_names = ['class1','class2']
print(classification_report(y_true, y_pred, target_names=target_names))

##Logistic Regression

In [ ]:
y_true = Y_test
y_pred = predictions_lr['prediction_label']
target_names = ['class1','class2']
print(classification_report(y_true, y_pred, target_names=target_names))

##Catboost

In [ ]:
y_true = df_Y
y_pred = predictions_catboost

target_names = ['class1','class2']

print(classification_report(y_true, y_pred, target_names=target_names))

##Load models

In [ ]:
CP_chunk_processed = preprocessf(P)
CP_chunk_processed.to_csv('/content/drive/My Drive/CPchunk_processed_main', index = False) 
df_chunk = pd.read_csv('/content/drive/My Drive/CPchunk_processed_main') 

##Load saved models

In [ ]:
import catboost as cb

lists = ['index', 'businesstags1005', 'businesstags1015', 'businesstags1017', 'bankscp400046', 'bankscp400234', 'bankscp505809', 'bankscp639217', 'bankscp983254']
for x in lists:
  df_chunk[x]=0

qda = load_model('/content/drive/My Drive/qda')
lr = load_model('/content/drive/My Drive/lr')
# catboost = load_model('/content/drive/My Drive/catboost')
model = CatBoostClassifier()
catboost=model.load_model("/content/drive/My Drive/catboost")


Predict the 1M chunk of customer profile

In [ ]:
qda_predictions = predict_model( qda,df_chunk.drop(columns = ['uuid', 'name', 'firstregion', 'rechargetypeidcp', 'secondprovince','secondregion']))
lr_predictions = predict_model( lr,df_chunk.drop(columns=['uuid', 'ussdid']))
catboost_predictions = model.predict(CP_chunk_processed)

#Select tourists of size 109,237

In [ ]:
qda_predictions[qda_predictions['prediction_score']==1].filter(items=['uuid', 'prediction_score']).shape

lr_predictions[lr_predictions['prediction_score']>=0.9755].filter(items=['uuid', 'prediction_score'])

lr_predictions[lr_predictions['prediction_score']>=0.9755].filter(items=['uuid', 'prediction_score']).to_csv('/content/drive/My Drive/campagin.csv')

#Select random of Customet profile of same size

In [ ]:
data = np.random.choice(lr_predictions.uuid, 109237)
savetxt('/content/drive/My Drive/campagin_random.csv', data, delimiter=',', fmt='%s')